<a href="https://colab.research.google.com/github/avrland/News-predict-word-count/blob/main/WordCountPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Count Predict
It's regression model trying to predict word count of news title from characters count. 
As data source we use [polishNewsTitleDatabase](https://github.com/avrland/polishNewsTitleDatabase), my scrapped base of over 100k news titles from polish sites.


In [ ]:
!pip install wget
import wget

In [3]:
def lineCounter(filename):
  file = open(filename, "r")
  line_count = 0
  for line in file:
      if line != "\n":
          line_count += 1
  return str(line_count)

url = 'https://raw.githubusercontent.com/avrland/polishNewsTitleDatabase/main/titles.txt'
filename = '/content/titles.txt'  
wget.download(url, filename)
with open(filename) as fn:  
  ln = fn.readline()
  lncnt = 0
  while lncnt < 5:
       print("Line {}: {}".format(lncnt, ln.strip()))
       ln = fn.readline()
       lncnt += 1
print("Database size: " + lineCounter(filename))

Line 0: Polka straciła 36 tys. zł: napastnik wykiwał zarówno ją jak i bank
Line 1: Chrome 86 na Androida pozwoli zaplanować pobieranie. Można już testować
Line 2: Poczta Polska i cyfrowa rewolucja. Identyfikacja RFID przyspieszy wysyłki
Line 3: GOG GALAXY 2.0 łączy siły z Epic Games Store. Jest wreszcie oficjalna integracja
Line 4: Android Auto ma problem – miesza okładki odtwarzanych albumów
Database size: 114952
